In [1]:
import pickle

# 加载.pkl文件
with open('game_data.pkl', 'rb') as f:
    data = pickle.load(f)

# 打印加载的数据
print(data)

         AppID                              Name  \
0      2525270                         The House   
1      2414640  Bite Size Terrors: Erobos Heaven   
2      2486670                  TD Tower Defense   
3      2304650                         The Scrap   
4      2519670                         Wind Love   
...        ...                               ...   
18555  1150090                  Learning Factory   
18556  1444570                  Fantasy Royal VR   
18557  1269950              Buddy Simulator 1984   
18558  1540140                   Tower of Portal   
18559  1533750  小小精灵塔防(Little elf tower defense)   

                                          About the game  \
0      ' The House ' is a short psychological horror ...   
1      Bite Size Terrors: Are Short Experimental Horr...   
2      Td Tower Defense is a fun tower defense shoote...   
3      The Scrap is an independent third-person shoot...   
4      Gameplay Wind Love - is a Japanese-style visua...   
...            

In [2]:
import pandas as pd
import numpy as np
import os
import pickle

In [3]:
# Data acquire: Kaggle

In [4]:
import pandas as pd

# Read the pkl file
#data = pd.read_pickle('C:/Users/19928/Desktop/Projects/Project-1/Code/game_data.pkl')

# Write the data from the pkl file to the csv file
#data.to_csv('C:/Users/19928/Desktop/Projects/Project-1/Code/game.csv', index=False, encoding='utf-8-sig')

PermissionError: [Errno 13] Permission denied: 'C:/Users/19928/Desktop/Projects/Project-1/Code/game.csv'

In [5]:
games_data=pd.read_csv('C:/Users/19928/Desktop/Projects/Project-1/Code/game.csv')
games_data.head()

,AppID,Name,About the game,Categories,Genres,Tags
0,2525270,The House,' The House ' is a short psychological horror ...,"Single-player,Steam Workshop","Adventure,Casual,Indie","Adventure,Casual,Action-Adventure,Walking Simu..."
1,2414640,Bite Size Terrors: Erobos Heaven,Bite Size Terrors: Are Short Experimental Horr...,"Single-player,Steam Achievements","Adventure,Indie","Horror,Adventure,Atmospheric,Psychological Hor..."
2,2486670,TD Tower Defense,Td Tower Defense is a fun tower defense shoote...,Single-player,"Action,Casual","Tower Defense,Casual,Sci-fi,Strategy,Combat,Re..."
3,2304650,The Scrap,The Scrap is an independent third-person shoot...,"Single-player,Steam Achievements","Action,Adventure,Indie","Action,Shooter,Third-Person Shooter,3D,Third P..."
4,2519670,Wind Love,Gameplay Wind Love - is a Japanese-style visua...,"Single-player,Steam Achievements,Captions avai...","Indie,Simulation","Hentai,Adventure,Dating Sim,Casual,Choices Mat..."


In [6]:
games_data=games_data[['AppID','Name','About the game','Categories','Genres','Tags']]

In [7]:
#Natural language processing
new_games_data=pd.DataFrame(games_data)
#字符串按空格进行分割
new_games_data['Genres']=new_games_data['Genres'].apply(lambda x:x.split())
new_games_data['Categories']=new_games_data['Categories'].apply(lambda x:x.split())
new_games_data['Tags']=new_games_data['Tags'].apply(lambda x:x.split())
new_games_data['About the game']=new_games_data['About the game'].apply(lambda x:x.split())
#移除三列中的空格
new_games_data['Categories']=new_games_data['Categories'].apply(lambda x:[i.replace(" ","") for i in x])
new_games_data['Genres']=new_games_data['Genres'].apply(lambda x:[i.replace(" ","") for i in x])
new_games_data['Tags']=new_games_data['Tags'].apply(lambda x:[i.replace(" ","") for i in x])
#形成新列
new_games_data['tags']=new_games_data['About the game'] + new_games_data['Categories'] + new_games_data['Genres'] + new_games_data['Tags']
#形成新Dataframe
new_df=new_games_data[['AppID','Name','tags']]
#小写并通过空格连接
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())
#词干提取
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\19928\AppData\Local\Temp\ipykernel_32096\2264921797.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))
C:\Users\19928\AppData\Local\Temp\ipykernel_32096\2264921797.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())
C:\Users\19928\AppData\Local\Temp\ipykernel_32096\2264921797.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [9]:
#计数向量,每个单词在每一个tags字段是否出现，是：1，否;0
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(new_df['tags']).toarray()
print(vectors.shape)

(18560, 5000)


In [10]:
#计算相似度矩阵
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)
def recommend(game):
    index = new_df[new_df['Name'] == game].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].Name)
print(similarity.shape)

(18560, 18560)


In [11]:
first_row = similarity[0]
print(first_row)

[1.         0.30961358 0.07803185 ... 0.36006607 0.15886771 0.17282528]


In [13]:
recommend('erozld')

Oh, Dungeon Master
Senpai and the Mysterious Dungeon
Into The Dark
Ouroboros Dungeon
Archmage Ricka


In [14]:
save_path = 'C:/Users/19928/Desktop/Projects/Project-1/Code/similarity.pkl'
pickle.dump(similarity, open(save_path, 'wb'))